In [31]:
%matplotlib inline
import h2o

from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
h2o.init()

H2O cluster uptime:,28 minutes 30 seconds 62 milliseconds
H2O cluster version:,3.7.0.3248
H2O cluster name:,H2O_started_from_python
H2O cluster total nodes:,1
H2O cluster total memory:,1.78 GB
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster healthy:,True
H2O Connection ip:,127.0.0.1
H2O Connection port:,54321


In [99]:
h2o.remove_all()
covtype_df = h2o.import_file("../data/covtype.full.csv")
#split the data as described above
train, valid, test = covtype_df.split_frame([0.6, 0.2], seed=1234)

#Prepare predictors and response columns
covtype_X = covtype_df.col_names[:-1]     #last column is Cover_Type, our desired response variable 
covtype_y = covtype_df.col_names[-1]    

train_df = train.as_data_frame(True)
valid_df = valid.as_data_frame(True)
test_df = test.as_data_frame(True)


Parse Progress: [##################################################] 100%


In [95]:
col = 'Elevation'


In [88]:
def place(value, breaks, len_breaks, range_cache):
    for k in range_cache:
        if value <breaks[k+1]:
            return k
    return len_breaks

In [96]:
%%time
col_col= train.as_data_frame(True)[col]
c, breaks = np.histogram(col_col, bins=20)
min_val = min(col_col)-1
max_val = max(col_col)+1
new_b = [min_val]
for i in xrange(19):
    if c[i] > 1000 and c[i+1] > 1000:
        new_b.append(breaks[i+1])
new_b.append(max_val)
nbl = len(new_b)-1
xr_nbl = range(nbl)
names = [col + '_' + str(x) for x in xrange(nbl)]
names.append("other")

new_col=[]


for val in col_col:
    new_col.append(names[place(val, new_b, nbl, xr_nbl)])

Wall time: 5.33 s


In [100]:
'''
Convenience function to cut a numeric column into intervals, creating a new categorical.
Uses h2o.hist to generate a histogram, with the buckets forming the categories of our new categorical.
Uses h2o.cut to do the split
Picks buckets based on training data, then applies the same classification to the test and validation sets

Assumes that train, test, valid will have the same histogram behavior.
'''
def cut_column(train_df, test_df, valid_df, col):
    only_col= train_df[col]                            #Isolate the column in question from the training frame
    counts, breaks = np.histogram(only_col, bins=20)   #Generate counts and breaks for our histogram
    min_val = min(col_col)-1                           #Establish min and max values
    max_val = max(col_col)+1
    
    new_b = [min_val]                                  #Redefine breaks such that each bucket has enough support
    for i in xrange(19):
        if c[i] > 1000 and c[i+1] > 1000:
            new_b.append(breaks[i+1])
    new_b.append(max_val)
    
    nbl = len(new_b)-1                                 #Cache bucket count and range(count) for performance reasons
    xr_nbl = range(nbl)
    
    
    names = [col + '_' + str(x) for x in xrange(nbl)]  #Generate names for buckets, these will be categorical names
    names.append("other")                              #Add 'other' bucket for everything not within min/max

    train_col=[]                                       #initialize new columns for categoricals
    test_col=[]
    valid_col=[]
    
    for val in only_col:
        train_col.append(names[place(val, new_b,       #populate categorical column for train
                                   nbl, xr_nbl)])
        
    for val in test_df[col]:
        test_col.append(names[place(val, new_b,        #populate categorical column for train
                                   nbl, xr_nbl)])
    for val in valid_df[col]:
        valid_col.append(names[place(val, new_b,       #populate categorical column for train
                                   nbl, xr_nbl)])
        
    train_df[col] = train_col
    test_df[col] = test_col
    valid_df[col] = valid_col

In [101]:
%%time
cut_column(train_df, test_df, valid_df, 'Elevation')

Wall time: 8.71 s


In [98]:
1792+483+193+155

2623